_**DELETE BEFORE PUBLISHING**_

_This is a template also containing the style guide for use cases. The styling uses the use-case css when uploaded to the website, which will not be visible on your local machine._

_Change any text marked with {} and delete any cells marked DELETE_

***

In [5]:
# DELETE BEFORE PUBLISHING
# This is just here so you can preview the styling on your local machine

from IPython.core.display import HTML
HTML("""
<style>
.usecase-title, .usecase-duration, .usecase-section-header {
    padding-left: 15px;
    padding-bottom: 10px;
    padding-top: 10px;
    padding-right: 15px;
    background-color: #0f9295;
    color: #fff;
}

.usecase-title {
    font-size: 1.7em;
    font-weight: bold;
}

.usecase-authors, .usecase-level, .usecase-skill {
    padding-left: 15px;
    padding-bottom: 7px;
    padding-top: 7px;
    background-color: #baeaeb;
    font-size: 1.4em;
    color: #121212;
}

.usecase-level-skill  {
    display: flex;
}

.usecase-level, .usecase-skill {
    width: 50%;
}

.usecase-duration, .usecase-skill {
    text-align: right;
    padding-right: 15px;
    padding-bottom: 8px;
    font-size: 1.4em;
}

.usecase-section-header {
    font-weight: bold;
    font-size: 1.5em;
}

.usecase-subsection-header, .usecase-subsection-blurb {
    font-weight: bold;
    font-size: 1.2em;
    color: #121212;
}

.usecase-subsection-blurb {
    font-size: 1em;
    font-style: italic;
}
</style>
""")

<div class="usecase-title">Parking Availability Near Places of Interest in Melbourne</div>

<div class="usecase-authors"><b>Authored by: </b> {Author/s}</div>

<div class="usecase-duration"><b>Duration:</b> {90} mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>{Intermediate}</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>{Python, and add any more skills needed}</div>
</div>

<div class="usecase-section-header">Scenario</div>

{Using User Story format, write a description of the problem you are trying to solve for this use case.}

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- {list the skills demonstrated in your use case}

<div class="usecase-section-header">{Heading for introduction or background relating to problem}</div>

{Write your introduction here. Keep it concise. We're not after "War and Peace" but enough background information to inform the reader on the rationale for solving this problem or background non-technical information that helps explain the approach. You may also wish to give information on the datasets, particularly how to source those not being imported from the client's open data portal.}



In [140]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.3/282.3 kB 4.1 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /opt/anaconda3/envs/MelbourneCityOpenData/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [173]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samihahaque/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [227]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/samihahaque/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [174]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [243]:
base_url='https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
dataset_id='landmarks-and-places-of-interest-including-schools-theatres-health-services-spor'


url=f'{base_url}{dataset_id}/exports/csv'
params={'select':'*','limit':-1,'lang':'en','timezone':'UTC'}

response=requests.get(url,params=params)

if response.status_code==200:
    url_content=response.content.decode('utf-8')
    places=pd.read_csv(StringIO(url_content),delimiter=';')
    print(places.head(10))
else:
    print(f'Request failed with status code {response.status_code}')

              theme           sub_theme                     feature_name  \
0  Place of Worship              Church                St Francis Church   
1  Place of Worship              Church                  St James Church   
2  Place of Worship              Church        St Mary's Anglican Church   
3  Place of Worship              Church                     Scots Church   
4  Place of Worship              Church      St Michael's Uniting Church   
5  Place of Worship              Church            Greek Orthodox Church   
6  Place of Worship              Church          North Melbourne Uniting   
7  Place of Worship              Church  South Yarra Presbyterian Church   
8  Place of Worship           Synagogue         East Melbourne Synagogue   
9         Transport  Transport Terminal                Port of Melbourne   

                          co_ordinates  
0  -37.8118847831837, 144.962422614541  
1  -37.8101281201969, 144.952468571683  
2  -37.8031663672997, 144.953761537074  

In [244]:
base_url='https://data.melbourne.vic.gov.au/api/explore/v2.1/catalog/datasets/'
dataset_id='on-street-parking-bays'


url=f'{base_url}{dataset_id}/exports/csv'
params={'select':'*','limit':-1,'lang':'en','timezone':'UTC'}

response=requests.get(url,params=params)

if response.status_code==200:
    url_content=response.content.decode('utf-8')
    parking_bays=pd.read_csv(StringIO(url_content),delimiter=';')
    print(parking_bays.head(10))
else:
    print(f'Request failed with status code {response.status_code}')

   roadsegmentid kerbsideid  \
0          22730        NaN   
1          22730        NaN   
2          20013       5701   
3          20013      23444   
4          22268        NaN   
5          22295        NaN   
6          22295        NaN   
7          22295        NaN   
8          21108        NaN   
9          20950        NaN   

                              roadsegmentdescription   latitude   longitude  \
0  Park Street between Mason Street and Randall P... -37.836245  144.982021   
1  Park Street between Mason Street and Randall P... -37.835800  144.982115   
2  Lonsdale Street between William Street and Kin... -37.814238  144.955451   
3  Lonsdale Street between William Street and Kin... -37.814271  144.955334   
4  Clowes Street between Anderson Street and Wals... -37.830568  144.984713   
5  Anderson Street between Domain Road and Acland... -37.833607  144.983763   
6  Anderson Street between Domain Road and Acland... -37.833657  144.983753   
7  Anderson Street between

This use case aims to evaluate the sufficiency of parking facilities around key places of interest in Melbourne. By analyzing the proximity and availability of on-street car parking meters and parking bays to various places of interest, the City of Melbourne can identify areas where parking infrastructure may need enhancement to support accessibility and convenience for visitors.

In [245]:
places

,theme,sub_theme,feature_name,co_ordinates
0,Place of Worship,Church,St Francis Church,"-37.8118847831837, 144.962422614541"
1,Place of Worship,Church,St James Church,"-37.8101281201969, 144.952468571683"
2,Place of Worship,Church,St Mary's Anglican Church,"-37.8031663672997, 144.953761537074"
3,Place of Worship,Church,Scots Church,"-37.8145687802664, 144.96855105335"
4,Place of Worship,Church,St Michael's Uniting Church,"-37.8143851324913, 144.969174036096"
...,...,...,...,...
237,Education Centre,School - Primary and Secondary Education,Melbourne Girls Grammar School,"-37.8315364518803, 144.985089428348"
238,Retail,Department Store,Myer,"-37.8135911985281, 144.963855087868"
239,Retail,Department Store,David Jones,"-37.8133127260638, 144.964373486798"
240,Health Services,Medical Services,Mercy Private Hospital,"-37.811896809802, 144.984435746587"


In [246]:
places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   theme         242 non-null    object
 1   sub_theme     242 non-null    object
 2   feature_name  242 non-null    object
 3   co_ordinates  242 non-null    object
dtypes: object(4)
memory usage: 7.7+ KB


In [247]:
places.shape

(242, 4)

In [248]:
places.isna().sum()

theme           0
sub_theme       0
feature_name    0
co_ordinates    0
dtype: int64

In [249]:
places['theme'].value_counts()

theme
Leisure/Recreation                      63
Place Of Assembly                       40
Place of Worship                        31
Transport                               26
Community Use                           21
Education Centre                        13
Mixed Use                               11
Health Services                         11
Office                                  11
Purpose Built                            4
Vacant Land                              3
Retail                                   3
Residential Accommodation                2
Specialist Residential Accommodation     1
Warehouse/Store                          1
Industrial                               1
Name: count, dtype: int64

In [250]:
places['sub_theme'].value_counts()

sub_theme
Informal Outdoor Facility (Park/Garden/Reserve)    37
Church                                             30
Railway Station                                    23
Art Gallery/Museum                                 19
Theatre Live                                       15
Major Sports & Recreation Facility                 14
Public Buildings                                   13
Office                                             11
Public Hospital                                     7
Retail/Office/Carpark                               5
Tertiary (University)                               4
Primary Schools                                     4
Outdoor Recreation Facility (Zoo, Golf Course)      4
Function/Conference/Exhibition Centre               4
Indoor Recreation Facility                          3
Police Station                                      3
Private Hospital                                    3
Retail/Office                                       3
Dwelling (House)  

In [251]:
for i in places['theme'].unique():
    if len(places.loc[places['theme']==i].value_counts())<5:
        print(places.loc[places['theme']==i])
        print("------------------------------------------")

           theme                               sub_theme  \
23   Vacant Land          Vacant Land - Undeveloped Site   
118  Vacant Land               Current Construction Site   
205  Vacant Land  Current Construction Site - Commercial   

                                feature_name  \
23   Melbourne International Karting Complex   
118                             Harbour Town   
205                   Railway Good Shed No 2   

                            co_ordinates  
23   -37.8310746380616, 144.913822928701  
118  -37.8139256326845, 144.938123825625  
205  -37.8211371302179, 144.951378883631  
------------------------------------------
             theme         sub_theme                            feature_name  \
33   Purpose Built  Film & RV Studio                    Central City Studios   
117  Purpose Built  Film & RV Studio  Channel 7 - Melbourne Broadcast Centre   
198  Purpose Built            Casino             Crown Entertainment Complex   
233  Purpose Built          Aqu

In [252]:
drop_themes=['Vacant Land','Purpose Built','Specialist Residential Accommodation','Residential Accommodation','Warehouse/Store','Industrial']
# places = places[~places['theme'].isin(drop_themes)]
places=places[~places['theme'].isin(drop_themes)]

In [253]:
print(places.shape)
places.head()

(230, 4)


,theme,sub_theme,feature_name,co_ordinates
0,Place of Worship,Church,St Francis Church,"-37.8118847831837, 144.962422614541"
1,Place of Worship,Church,St James Church,"-37.8101281201969, 144.952468571683"
2,Place of Worship,Church,St Mary's Anglican Church,"-37.8031663672997, 144.953761537074"
3,Place of Worship,Church,Scots Church,"-37.8145687802664, 144.96855105335"
4,Place of Worship,Church,St Michael's Uniting Church,"-37.8143851324913, 144.969174036096"


In [255]:
co_ord_list = places['co_ordinates'].str.split(", ", expand=True)
places.loc[:, 'latitude'] = pd.to_numeric(co_ord_list[0])
places.loc[:, 'longitude'] = pd.to_numeric(co_ord_list[1])

In [256]:
places.drop(columns='co_ordinates',axis=1,inplace=True)
places.reset_index(drop=True, inplace=True)

/var/folders/vq/x79kmsys0093sdv6ft95m8k00000gn/T/ipykernel_56569/2915858193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  places.drop(columns='co_ordinates',axis=1,inplace=True)


In [257]:
for i in places.columns[:-2]:
    places.loc[:,i]=places[i].str.lower()
places.head()

,theme,sub_theme,feature_name,latitude,longitude
0,place of worship,church,st francis church,-37.811885,144.962423
1,place of worship,church,st james church,-37.810128,144.952469
2,place of worship,church,st mary's anglican church,-37.803166,144.953762
3,place of worship,church,scots church,-37.814569,144.968551
4,place of worship,church,st michael's uniting church,-37.814385,144.969174


In [274]:
corpus=places.loc[:,'theme']+" "+places.loc[:,'sub_theme']+" "+places.loc[:,'feature_name']
places.loc[:,'corpus']=corpus

In [275]:
pattern = r'[^a-zA-Z0-9\s]'
places.loc[:,'corpus'] = places['corpus'].str.replace(pattern, ' ', regex=True)
# places.loc[:,'corpus']=places['corpus'].str.replace(r'\s+', ' ', regex=True).str.strip()

In [278]:
places.loc[:,'corpus_token']=places['corpus'].apply(word_tokenize)

In [280]:
stop_words=stopwords.words('english')
lst=[]
for i in range(len(places['corpus_token'])):
    st=""
    for j in places.loc[i,'corpus_token']:
        if j not in stop_words:
            st+=j+" "
    
    lst.append(st)

In [284]:
places.loc[:,'corpus_clean']=lst
places

,theme,sub_theme,feature_name,latitude,longitude,corpus,corpus_token,corpus_clean
0,place of worship,church,st francis church,-37.811885,144.962423,place of worship church st francis church,"[place, of, worship, church, st, francis, church]",place worship church st francis church
1,place of worship,church,st james church,-37.810128,144.952469,place of worship church st james church,"[place, of, worship, church, st, james, church]",place worship church st james church
2,place of worship,church,st mary's anglican church,-37.803166,144.953762,place of worship church st mary s anglican church,"[place, of, worship, church, st, mary, s, angl...",place worship church st mary anglican church
3,place of worship,church,scots church,-37.814569,144.968551,place of worship church scots church,"[place, of, worship, church, scots, church]",place worship church scots church
4,place of worship,church,st michael's uniting church,-37.814385,144.969174,place of worship church st michael s uniting c...,"[place, of, worship, church, st, michael, s, u...",place worship church st michael uniting church
...,...,...,...,...,...,...,...,...
225,education centre,school - primary and secondary education,melbourne girls grammar school,-37.831536,144.985089,education centre school primary and secondar...,"[education, centre, school, primary, and, seco...",education centre school primary secondary educ...
226,retail,department store,myer,-37.813591,144.963855,retail department store myer,"[retail, department, store, myer]",retail department store myer
227,retail,department store,david jones,-37.813313,144.964373,retail department store david jones,"[retail, department, store, david, jones]",retail department store david jones
228,health services,medical services,mercy private hospital,-37.811897,144.984436,health services medical services mercy private...,"[health, services, medical, services, mercy, p...",health services medical services mercy private...


===============================================================================================================

In [30]:
parking_bays

,roadsegmentid,roadsegmentdescription,latitude,longitude,lastupdated
0,22730,Park Street between Mason Street and Randall P...,-37.836245,144.982021,2022-08-31
1,22730,Park Street between Mason Street and Randall P...,-37.835800,144.982115,2022-08-31
2,20013,Lonsdale Street between William Street and Kin...,-37.814238,144.955451,2023-10-02
3,20013,Lonsdale Street between William Street and Kin...,-37.814271,144.955334,2023-10-02
4,22268,Clowes Street between Anderson Street and Wals...,-37.830568,144.984713,2022-08-31
...,...,...,...,...,...
19157,22492,Alexandra Avenue between Swan Street Bridge an...,-37.827226,144.980441,2022-08-31
19158,22492,Alexandra Avenue between Swan Street Bridge an...,-37.827380,144.981004,2022-08-31
19159,22492,Alexandra Avenue between Swan Street Bridge an...,-37.826361,144.979266,2022-08-31
19160,22492,Alexandra Avenue between Swan Street Bridge an...,-37.826780,144.979662,2022-08-31


In [26]:
parking_bays.shape

(19162, 5)

In [22]:
parking_bays.isna().sum()

roadsegmentid                 0
kerbsideid                14149
roadsegmentdescription        0
latitude                      0
longitude                     0
lastupdated                   0
dtype: int64

In [25]:
parking_bays.drop(columns='kerbsideid',inplace=True)
parking_bays.isna().sum()

roadsegmentid             0
roadsegmentdescription    0
latitude                  0
longitude                 0
lastupdated               0
dtype: int64